In [9]:
# %%
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.preprocessing import LabelEncoder
from scipy.spatial.distance import cdist
from tqdm import tqdm

In [10]:

dist_df = pd.read_csv("DATA/HB/dist_df_100time.csv" )
angle_df = pd.read_csv("DATA/HB/angle_df_100time.csv" )

In [11]:
dist_df

,src,dst,time,distance,src_mol,dst_mol,nh_id,label
0,O5_1,N11_1,255000,4.435484,1,1,1,1
1,O5_1,N23_1,255000,5.677049,1,1,2,1
2,O5_1,N35_1,255000,6.328246,1,1,3,1
3,O5_1,N55_1,255000,6.667776,1,1,4,1
4,O5_1,N67_1,255000,7.467732,1,1,5,1
...,...,...,...,...,...,...,...,...
12873595,O2_14,N727_4,255099,79.479108,14,4,212,2
12873596,O2_14,N739_4,255099,75.781733,14,4,213,2
12873597,O2_14,N759_4,255099,75.197622,14,4,214,2
12873598,O2_14,N771_4,255099,84.087391,14,4,215,2


In [12]:
angle_df

,src,dst,time,angle,src_mol,dst_mol,nh_id,label
0,O5_1,H9_1,255000,61.040505,1,1,1,1
1,O5_1,H19_1,255000,73.524710,1,1,2,1
2,O5_1,H29_1,255000,88.194570,1,1,3,1
3,O5_1,H46_1,255000,120.540965,1,1,4,1
4,O5_1,H56_1,255000,115.887955,1,1,5,1
...,...,...,...,...,...,...,...,...
12873595,O2_14,H611_4,255099,166.764098,14,4,212,2
12873596,O2_14,H621_4,255099,62.206380,14,4,213,2
12873597,O2_14,H638_4,255099,99.980679,14,4,214,2
12873598,O2_14,H648_4,255099,137.762045,14,4,215,2


In [13]:
# Combine src and dst values for consistent mapping
combined_values = pd.concat([
    dist_df['src'], dist_df['dst'], 
    angle_df['src'], angle_df['dst']
])

# Use factorize to assign numeric indices starting from 0
numeric_indices, _ = pd.factorize(combined_values)

# Add 1 to ensure indices start from 1
numeric_indices += 1

# Map src and dst directly to numeric indices using factorized output
mapping = pd.Series(numeric_indices, index=combined_values).to_dict()

# Replace src and dst with numeric indices starting from 1
for df in [dist_df, angle_df]:
    df['src'] = df['src'].map(mapping)
    df['dst'] = df['dst'].map(mapping)

# Display the filtered DataFrames
print("Filtered Distance DataFrame:")
print(dist_df)

print("\nFiltered Angle DataFrame:")
print(angle_df)

Filtered Distance DataFrame:
          src  dst    time   distance  src_mol  dst_mol  nh_id  label
0           1  597  255000   4.435484        1        1      1      1
1           1  598  255000   5.677049        1        1      2      1
2           1  599  255000   6.328246        1        1      3      1
3           1  600  255000   6.667776        1        1      4      1
4           1  601  255000   7.467732        1        1      5      1
...       ...  ...     ...        ...      ...      ...    ...    ...
12873595  596  808  255099  79.479108       14        4    212      2
12873596  596  809  255099  75.781733       14        4    213      2
12873597  596  810  255099  75.197622       14        4    214      2
12873598  596  811  255099  84.087391       14        4    215      2
12873599  596  812  255099  83.915797       14        4    216      2

[12873600 rows x 8 columns]

Filtered Angle DataFrame:
          src   dst    time       angle  src_mol  dst_mol  nh_id  label
0  

In [15]:
# Function to process DataFrame and save it to a CSV file
def process_and_save_dataframe(df, filename):
    """
    Process the input DataFrame to add 'ext_roll' and 'idx' columns.
    Convert all columns to integers before saving.
    Save the processed DataFrame to a specified CSV file.

    Parameters:
    - df (pd.DataFrame): Input DataFrame
    - filename (str): Output file path
    """
    num_rows = len(df)

    # Initialize 'ext_roll' column with zeros
    df['ext_roll'] = 0

    # Assign 1 to the middle 15% rows and 2 to the last 15% rows
    df.loc[int(num_rows * 0.7):int(num_rows * 0.85) - 1, 'ext_roll'] = 1
    df.loc[int(num_rows * 0.85):, 'ext_roll'] = 2

    # Insert an 'idx' column at the beginning
    df.insert(0, 'idx', range(len(df)))

    # Reindex and retain only the required columns
    df = df[['idx', 'src', 'dst', 'time', 'label', 'ext_roll', 'nh_id']]

    # Convert all columns to integers
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Save the updated DataFrame to the specified CSV file
    df.to_csv(filename, index=False)
    print(f"Saved updated DataFrame to {filename}")

    return df

In [16]:
# Process and save distance DataFrame
final_dist_df = process_and_save_dataframe(dist_df, 'DATA/HB/test_edges1.csv')

# Process and save angle DataFrame
final_angle_df = process_and_save_dataframe(angle_df, 'DATA/HB/test_edges2.csv')

# Display the first few rows of both DataFrames
print("Updated Distance DataFrame:")
print(final_dist_df)

print("\nUpdated Angle DataFrame:")
print(final_angle_df)

/tmp/ipykernel_1048160/2464678129.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
/tmp/ipykernel_1048160/2464678129.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
/tmp/ipykernel_1048160/2464678129.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

Saved updated DataFrame to DATA/HB/test_edges1.csv


/tmp/ipykernel_1048160/2464678129.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
/tmp/ipykernel_1048160/2464678129.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
/tmp/ipykernel_1048160/2464678129.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

Saved updated DataFrame to DATA/HB/test_edges2.csv
Updated Distance DataFrame:
               idx  src  dst    time  label  ext_roll  nh_id
0                0    1  597  255000      1         0      1
1                1    1  598  255000      1         0      2
2                2    1  599  255000      1         0      3
3                3    1  600  255000      1         0      4
4                4    1  601  255000      1         0      5
...            ...  ...  ...     ...    ...       ...    ...
12873595  12873595  596  808  255099      2         2    212
12873596  12873596  596  809  255099      2         2    213
12873597  12873597  596  810  255099      2         2    214
12873598  12873598  596  811  255099      2         2    215
12873599  12873599  596  812  255099      2         2    216

[12873600 rows x 7 columns]

Updated Angle DataFrame:
               idx  src   dst    time  label  ext_roll  nh_id
0                0    1   813  255000      1         0      1
1         